In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO
warnings.filterwarnings('ignore')

D:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [4]:
def crop(image, boxes, scores, classes, all_classes):
    """crop the box of person on the image.
    """
    crop_image = None
    for box, score, cl in zip(boxes, scores, classes):
        if all_classes[cl] == "person":
            x, y, w, h = box
            
            top = max(0, np.floor(x + 0.5).astype(int))
            left = max(0, np.floor(y + 0.5).astype(int))
            right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
            bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
        
            crop_image = image[left:bottom, top:right]
            
    return crop_image

In [5]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images and crop 

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)
    crop_image = None

    boxes, classes, scores = yolo.predict(pimage, image.shape)

    if boxes is not None:
        crop_image = crop(image, boxes, scores, classes, all_classes)
        
    return crop_image

In [7]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)
m = 0

rootdir = 'images/test'  # 指明被遍历的文件夹
outdir = 'images/crop/' # 存放裁剪后图片的文件夹

for (root, dirs, files) in os.walk(rootdir):
    if files:
        for f in files:
            path = os.path.join(root, f)
            image = cv2.imread(path)
            crop_image = detect_image(image, yolo, all_classes)
            if crop_image is not None:
                cv2.imwrite(outdir + "crop_" + f, crop_image)
                m += 1
                if m%20 == 0:
                    print('{} images have preprocessed.'.format(m))

D:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


2 images have preprocessed.
4 images have preprocessed.
